<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/data_preprocessing/optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDC(Golden Dead Cross)

In [ ]:
!pip install backtrader

     |████████████████████████████████| 410 kB 37.2 MB/s 


In [ ]:
pip install finance-datareader

In [ ]:
pip install backtesting

     |████████████████████████████████| 175 kB 32.6 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173823 sha256=7709d918275fc337fadd830fe5cb13f868794dd550dcb11354494bdcdcb981be
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting


In [ ]:
pip install deap

     |████████████████████████████████| 160 kB 25.7 MB/s 


In [ ]:
#talib 설치
!wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null
!tar xvzf ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null

import os

os.chdir('ta-lib')  

!./configure --prefix=/usr 2>&1 > /dev/null
!make 2>&1 > /dev/null
!make install 2>&1 > /dev/null

os.chdir('../')

!pip install TA-Lib 2>&1 > /dev/null

./configure: line 4354: /usr/bin/file: No such file or directory
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
ar: `u' modifier ignored since `D' is the default (see `U')
gen_code.c: In function ‘printFuncHeaderDoc’:
gen_code.c:3456:4: warning: format not a string literal and no format arguments [-Wformat-security]
    fprintf( out, prefix );
    ^~~~~~~


In [ ]:
import re
import json
import talib
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, date, timedelta
from google.colab import drive
import backtrader as bt
from IPython.display import display, Image
import backtrader.feeds as btfeeds
from backtesting import Strategy
from backtesting import Backtest
from deap import base, creator, tools, algorithms
import random
import seaborn as sns
from tqdm import trange
import time

pd.options.mode.chained_assignment = None  # default='warn'

/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/[Shared]K-Project/data/'

Mounted at /content/drive


In [ ]:
## Load pickle - fullcode 목록
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
    
# 상장폐지 상태 제외시키기
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']

# fullcode열을 list로 변경
fullcode_list = list(fullcode_filter['full_code'])
fullcode_list0 = fullcode_list[:2]
# fullcode_list0 = fullcode_list[260:520]
# fullcode_list0 = fullcode_list[520:780]
# fullcode_list0 = fullcode_list[780:1040]
# fullcode_list0 = fullcode_list[1040:1300]
# fullcode_list0 = fullcode_list[1300:1560]
# fullcode_list0 = fullcode_list[1560:1820]
# fullcode_list0 = fullcode_list[1820:2080]
# fullcode_list0 = fullcode_list[2080:2340]
# fullcode_list0 = fullcode_list[2340:]
print(len(fullcode_list0))

2


In [ ]:
with open(data_path + 'initial_data1.json',"r") as json_file:
    json_data = json.load(json_file) 

In [ ]:
result_dict = {}

for j in range(len(fullcode_list0)): 

  print(j)
  print(fullcode_list0[j])
  
  # try:
  # backtest 데이터 가공

  df = pd.read_json(json_data[fullcode_list0[j]], orient ='index') # 첫번째 키값으로 데이터프레임 변환
  df = df.transpose()

  #시간순 재정렬
  df = df.sort_values(by=['TRD_DD'])
  df.reset_index(drop=True,inplace=True)
  df['TRD_DD'] = pd.to_datetime(df['TRD_DD']) #datetime변환

  #인풋 데이터 모양 맞춰주기(backtest에 들어갈 데이터 모양)
  df_bt = df[['TRD_DD','TDD_OPNPRC','TDD_HGPRC','TDD_LWPRC','TDD_CLSPRC', 'ACC_TRDVOL']]
  df_bt['TRD_DD'] = pd.to_datetime(df_bt['TRD_DD'])
  df_bt.rename(columns={'TRD_DD':'Date', 'TDD_OPNPRC':'Open', 'TDD_HGPRC':'High','TDD_LWPRC':'Low','TDD_CLSPRC':'Close', 'ACC_TRDVOL':'Volume'}, inplace=True)
  df_bt.set_index('Date',drop=True,inplace=True)

  #데이터프레임 콤마(,) 제거 그리고 타입 소수로 변환
  df_bt['Open'] = df_bt['Open'].str.replace(',','').astype('float')
  df_bt['High'] = df_bt['High'].str.replace(',','').astype('float')
  df_bt['Low'] = df_bt['Low'].str.replace(',','').astype('float')
  df_bt['Close'] = df_bt['Close'].str.replace(',','').astype('float')
  df_bt['Volume'] = df_bt['Volume'].str.replace(',','').astype('float')

  class SmaCross1(bt.Strategy):
    params = dict(
      pfast=50, # period for the fast moving average
      pslow=200 # period for the slow moving average 
      ) 
    
    def __init__(self):
      sma1 = bt.ind.SMA(period = self.p.pfast) # fast moving average 
      sma2 = bt.ind.SMA(period = self.p.pslow) # slow moving average 
      self.crossover = bt.ind.CrossOver(sma1, sma2) # crossover signal 

    def next(self): 
      if not self.position: # not in the market 
        if self.crossover > 0: # if fast crosses slow to the upside 
          close = self.data.close[0] # 종가 값 
            
          size = int(self.broker.getcash() / close) # 최대 구매 가능 개수 
          self.buy(size=size) # 매수 size = 구매 개수 설정 
      elif self.crossover < 0: # in the market & cross to the downside 
          self.close() # 매도

  random.seed(3)

  PARAM_NAMES = ["pfast", "pslow"]

  NGEN = 5  # 알고리즘 5번 반복.
  NPOP = 100 #인구 초기
  CXPB = 0.5  #교차 전략 
  MUTPB = 0.3  #돌연변이 전략.


  #최소fintness 설정 (fitness값이 작을수록 좋도록 설정)
  creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
  creator.create('Individual', list, fitness=creator.FitnessMin)

  # creator.create("FitnessMax", base.Fitness, weights=(1.0,))
  # creator.create("Individual", list, fitness=creator.FitnessMax)

  def evaluate(individual, plot=False, log=False):

    strategy_params = {k: v for k, v in zip(PARAM_NAMES, individual)}

    cerebro = bt.Cerebro(stdstats=False)

    data = bt.feeds.PandasData(dataname = df_bt, name = i)

    cerebro.adddata(data)

    initial_capital = 1000000
    cerebro.broker.setcash(initial_capital)

    cerebro.addstrategy(SmaCross1, **strategy_params)

    cerebro.addanalyzer(bt.analyzers.DrawDown)

    cerebro.broker.setcommission(commission=0.0025, margin=False)  #수수료 설정

    strats = cerebro.run()

    profit = cerebro.broker.getvalue() - initial_capital
    
    if profit == 0:
      return [np.inf]

    # max_dd = strats[0].analyzers.drawdown.get_analysis()["max"]["moneydown"] # max.moneydown - max drawdown value in monetary units
    # fitness = profit / (max_dd if max_dd > 0 else 1)
    fitness = round(1 / profit, 15)

    if log:
      print(f"Starting Portfolio Value: {initial_capital:,.2f}")
      print(f"Final Portfolio Value:  {cerebro.broker.getvalue():,.2f}")
      print(f"Total Profit:       {profit:,.2f}")
      print(f"Maximum Drawdown:     {max_dd:,.2f}")
      print(f"Profit / Max DD:     {fitness}")

    # if plot:
      # cerebro.plot()

    return [fitness]

  toolbox = base.Toolbox()
  toolbox.register("indices", random.sample, range(NPOP), NPOP)

  # crossover strategy
  toolbox.register("mate", tools.cxUniform, indpb=CXPB)
  # mutation strategy
  toolbox.register("mutate", tools.mutUniformInt, low=1, up=151, indpb=0.2)
  # selection strategy
  toolbox.register("select", tools.selTournament, tournsize=3)
  # fitness function
  toolbox.register("evaluate", evaluate)


  # definition of an individual & a population
  toolbox.register("attr_sma1", random.randint, 1, 100)
  toolbox.register("attr_sma2", random.randint, 151, 251) 
  toolbox.register(
    "individual",
    tools.initCycle,
    creator.Individual,
    (
      toolbox.attr_sma1,
      toolbox.attr_sma2,

    ),
  )

  toolbox.register("population", tools.initRepeat, list, toolbox.individual)

  mean = np.ndarray(NGEN)
  best = np.ndarray(NGEN)
  hall_of_fame = tools.HallOfFame(maxsize=3)

  t = time.perf_counter()
  pop = toolbox.population(n=NPOP)
  for g in trange(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
      if random.random() < CXPB:
        toolbox.mate(child1, child2)
        del child1.fitness.values
        del child2.fitness.values

    # Apply mutation on the offspring
    for mutant in offspring:
      if random.random() < MUTPB:
        toolbox.mutate(mutant)
        del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
      ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring
    hall_of_fame.update(pop)
    print(
      "HALL OF FAME:\n"
      + "\n".join(
        [
          f"  {_}: {ind}, Fitness: {ind.fitness.values[0]}"
          for _, ind in enumerate(hall_of_fame)
        ]
      )
    )

    fitnesses = [
      ind.fitness.values[0] for ind in pop if not np.isinf(ind.fitness.values[0])
    ]
    mean[g] = np.mean(fitnesses)
    best[g] = np.max(fitnesses)

  end_t = time.perf_counter()
  print(f"Time Elapsed: {end_t - t:,.2f}")

  # fig, ax = plt.subplots(sharex=True, figsize=(16, 9))

  # sns.lineplot(x=range(NGEN), y=mean, ax=ax, label="Average Fitness Score")
  # sns.lineplot(x=range(NGEN), y=best, ax=ax, label="Best Fitness Score")
  # ax.set_title("Fitness Score")
  # ax.set_xticks(range(NGEN))
  # ax.set_xlabel("Iteration")

  # plt.tight_layout()
  # plt.show()
  
  # 최적의 파라미터 값 출력
  OPTIMISED_STRATEGY_PARAMS = {
    k: v for k, v in zip(PARAM_NAMES, hall_of_fame[0])}
  op_params = list(OPTIMISED_STRATEGY_PARAMS.values())
  print('**파라미터 값 (try): ', op_params)
  
  # except:
  #   op_params = [50, 200]
  #   print('**파라미터 값 (except): ', op_params)


  # 데이터 불러와서 가공
  df = pd.read_json(json_data[fullcode_list0[j]], orient ='index') 
  read_df = df.transpose()
  read_df['TDD_CLSPRC'] = read_df['TDD_CLSPRC'].str.replace(',','').astype('float')
  read_df['pfast'] = talib.MA(read_df['TDD_CLSPRC'], timeperiod = op_params[0], matype=0)
  read_df['pslow'] = talib.MA(read_df['TDD_CLSPRC'], timeperiod = op_params[1], matype=0)
  read_df["GDC_sig"] = ""
  
  # 매도, 매수 전략 설정 후 GDC_sig 열 추가
  first_cross = 0 
  for i in range(0, len(read_df)):
    if read_df['pfast'][i] < read_df['pslow'][i] and first_cross == 0:
      # print('Death cross on day', df['TRD_DD'][i], ':expect the price to continue to fall (매도)')
      read_df['GDC_sig'][i] = -1
      first_cross=1
    elif read_df['pfast'][i] > read_df['pslow'][i] and first_cross ==1:
      # print('Golden cross on day', df['TRD_DD'][i], ':expect the price to continue to rise (매수)')
      first_cross=0
      read_df['GDC_sig'][i] = 1
    else:
      read_df['GDC_sig'][i] = 0
  
  result = read_df.drop(['pfast', 'pslow'], axis='columns')
  # print(result)

  result_js = result.to_json(orient = 'columns') # 데이터프레임을 json 형태로 변환
  result_dict[fullcode_list0[j]] = result_js # fullcode가 key값이 되고 json이 value값이 됨

with open('/content/drive/MyDrive/[Shared]K-Project/data/test1.json', 'w') as f:
    json.dump(result_dict, f)


0
KR7060310000


/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
 20%|██        | 1/5 [03:06<12:27, 186.82s/it]

HALL OF FAME:
  0: [3, 193], Fitness: -3.1397568572e-05
  1: [67, 200], Fitness: -1.7942111571e-05
  2: [47, 233], Fitness: -1.7858653667e-05


 40%|████      | 2/5 [05:04<07:18, 146.32s/it]

HALL OF FAME:
  0: [3, 193], Fitness: -3.1397568572e-05
  1: [67, 200], Fitness: -1.7942111571e-05
  2: [47, 233], Fitness: -1.7858653667e-05


...

HALL OF FAME:
  0: [34, 129], Fitness: -0.00013092432574
  1: [142, 181], Fitness: -0.000108868421626
  2: [26, 232], Fitness: -9.130981649e-05


 80%|████████  | 4/5 [02:55<00:41, 41.91s/it]

HALL OF FAME:
  0: [34, 129], Fitness: -0.00013092432574
  1: [78, 192], Fitness: -0.000109961705836
  2: [142, 181], Fitness: -0.000108868421626


100%|██████████| 5/5 [03:30<00:00, 42.16s/it]

HALL OF FAME:
  0: [34, 192], Fitness: -0.000281850914959
  1: [34, 129], Fitness: -0.00013092432574
  2: [78, 192], Fitness: -0.000109961705836
Time Elapsed: 210.79
**파라미터 값 (try):  [34, 192]


In [ ]:
# 제대로 저장되었는지 테스트
with open('/content/drive/MyDrive/[Shared]K-Project/data/test1.json',"r") as json_file:
    json_data = json.load(json_file) # json file open

df = pd.read_json(json_data['KR7060310000'], orient ='index') # 첫번째 키값으로 데이터프레임 변환
json_df = df.transpose()
print(json_df)

          TRD_DD  ISU_CD  ISU_NM TDD_CLSPRC  ...   PBR DPS DVD_YLD GDC_sig
0     2021/12/13  060310      3S       3405  ...  4.30   0    0.00       0
1     2021/12/10  060310      3S       3430  ...  4.33   0    0.00       0
2     2021/12/09  060310      3S       3490  ...  4.41   0    0.00       0
3     2021/12/08  060310      3S       3465  ...  4.38   0    0.00       0
4     2021/12/07  060310      3S       3465  ...  4.38   0    0.00       0
...          ...     ...     ...        ...  ...   ...  ..     ...     ...
4853  2002/04/29  060310  삼에스코리아       6780  ...     -   -       -       0
4854  2002/04/26  060310  삼에스코리아       7700  ...     -   -       -       0
4855  2002/04/25  060310  삼에스코리아       8750  ...     -   -       -       0
4856  2002/04/24  060310  삼에스코리아       8510  ...     -   -       -       0
4857  2002/04/23  060310  삼에스코리아       7600  ...     -   -       -       0

[4858 rows x 16 columns]


In [ ]:
# json 파일 합치는 방법
# https://great-woman-hoseung.tistory.com/6